In [1]:
from final_report import tokenize_doc, mask_tokens, train_mle, train_laplace, train_lidstone, train_stupid_backoff, test_model

# Abstract

In this project, we endeavored to create and experiment with text prediction models using N-Grams. MLE, Laplace, Lidstone, and Stupid Backoff models were trained and tested in order to quantify the accuracy of predictions and number of keystrokes said prediction would save a hypothetical user. Our 0.75-alpha Stupid Backoff model performed the best, although the results were not drastically different across models.

# Introduction

From phone keyboards to email-assisting Chrome extensions, word prediction is increasingly utilized in technological endeavors. As mobile devices continue to become more accessible to the average person, quick and quality text prediction is becoming a critical part of this rollout. Additionally, in the field of augmentative and alternative communications (AAC), word prediction can drastically decrease the amount of required keystrokes for a user with communication disabilities using a letter-based device, increasing communication speed and ease.
 
Through this report, we will tackle the topic of building such a piece of software to predict words so we can deepen our understanding of the technology and its utility. N-Gram are specifically useful as they can "remember" recent words and make inferences off them. By using different versions of N-Gram models, we vary our approach as to identify the ideal method of constructing the model. In our experiments, we test Machine Likelihood Estimator (MLE), Laplace, Lidstone, and Stupid Backoff models. We also varied the value of alpha in alpha-smoothing where applicable.

We used two different metrics to measure efficacy in our models. Exact accuracy measured the probability that the model was able to correctly predict the next word. Keystrokes saved measured the amount of characters left in the word when the word is guessed. Correctly predicting a word on its first letter would result in a +2 keystrokes saved count over  its third.

# Related Work

# Data

To create our prediction models, we used a collection of 100,000 Amazon reviews across a multitide of products and reviewers gathered by the Stanford Network Analysis Project. Each review included it's polarity (positive or negative), a title, and the text of the review. The polarity of each review was determined through the star ratings given, 1 and 2 being negative with 4 and 5 being positive. Reviews of 3 stars are not included.

Link: https://www.kaggle.com/kritanjalijain/amazon-reviews

# Method

## Tokenization & Normalization

Tokenization and normalization is very important as we need to ensure that our NLP model is not skewed by unclean data. Our first step in tokenization is to separate our training Amazon reviews into a list of sentence tokens using NLTK's sentence tokenizer. Finally, separate this list of sentence tokens into a list of lists of word tokens using NLTK's TweetTokenizer. As for normalization, we chose to lowercase all the word tokens as to get more meaningful results not altered by capitalization.

In [2]:
sentence_tokens = []
with open('../data/sample_train.csv') as csv_file:
    print('Tokenizing and normalizing training data...', end=' ')
    sentence_tokens = tokenize_doc(csv_file)
    print('Done')

Tokenizing and normalizing training data... Done


In [3]:
import statistics

print('Total Training Sentence Tokens:', len(sentence_tokens))
print('Average Number of Training Word Tokens per Training Sentence Token:', statistics.mean(map(lambda e: len(e), sentence_tokens)))
print('Example Training Sentence Token:', sentence_tokens[0])

Total Training Sentence Tokens: 165696
Average Number of Training Word Tokens per Training Sentence Token: 18.35504779837775
Example Training Sentence Token: ['this', 'sound', 'track', 'was', 'beautiful', '!']


## N-Gram Models

To begin, we first chose to model our problem uaing several N-Gram models. This is a rather simple approach to word prediction since we can store an arbitrary number of N-Grams and predict words based on the previous N - 1 words. To be more specific, we limited our N-Gram models to trigrams, bigrams, and unigrams which means we can predict words based on 2 or less previous words. The predicted word is the N-Gram that has the highest probability with that word as the last item and the previous items being the previous words, if any. The creation and preparation of such N-Grams for our models can be handled by NLTK's Everygram Preprocessor. The models come from NLTK's LanguageModels which we decided to use because of the ease of use in terms of input and output for our tokens.

### Maximum Likelihood Estimator (MLE)

The MLE model serves as the basis for our N-Gram modeling. It utilizes the algorithm described above without any smoothing or extra preparation. The model's only concern is the raw likelihoods for the word predictions.

In [4]:
print('Training MLE model...', end=' ')
mle = train_mle(sentence_tokens)
print('Done')

Training MLE model... Done


### Laplace

The Laplace model utilizes the MLE model while also implemeting add-1 smoothing. This leads to more accurate word probabilities in general since we can assign non-zero probabilities to unseen words. We are using this model as a direct comparison to the MLE model.

In [5]:
print('Training Laplace model...', end=' ')
laplace = train_laplace(sentence_tokens)
print('Done')

Training Laplace model... Done


### Lidstone

The Lidstone model is the same as the Laplace model but instead of add-1 smoothing, we can specify the amount of smoothing. We chose to create three different Lidstone models, initialized with add-0.25 smoothing, add-0.5 smoothing, and add-0.75 smoothing, respectively. We are using this model to demonstrate how smoothing affects the word predictions.

In [6]:
print('Training Lidstone models...')

print('Training Lidstone (gamma=0.25) model...', end=' ')
lidstone_25 = train_lidstone(sentence_tokens, 0.25)
print('Done')

print('Training Lidstone (gamma=0.5) model...', end=' ')
lidstone_50 = train_lidstone(sentence_tokens, 0.5)
print('Done')

print('Training Lidstone (gamma=0.75) model...', end=' ')
lidstone_75 = train_lidstone(sentence_tokens, 0.75)
print('Done')

Training Lidstone models...
Training Lidstone (gamma=0.25) model... Done
Training Lidstone (gamma=0.5) model... Done
Training Lidstone (gamma=0.75) model... Done


### Stupid Backoff

The Stupid Backoff model utilizes the MLE model while also providing the ability to scale lower order probabilities. The downside of this is that it is not a true probability distribution. We chose to create three different Stupid Backoff models, initialized with 0.25, 0.5, and 0.75, respectively. We are using this model to determine at what degree lower order probabilities affect the word predictions

In [7]:
print('Training Stupid Backoff models...')

print('Training Stupid Backoff (alpha=0.25) model...', end=' ')
stupid_backoff_25 = train_stupid_backoff(sentence_tokens, 0.25)
print('Done')

print('Training Stupid Backoff (alpha=0.25) model...', end=' ')
stupid_backoff_50 = train_stupid_backoff(sentence_tokens, 0.5)
print('Done')

print('Training Stupid Backoff (alpha=0.25) model...', end=' ')
stupid_backoff_75 = train_stupid_backoff(sentence_tokens, 0.75)
print("Done")

Training Stupid Backoff models...
Training Stupid Backoff (alpha=0.25) model... Done
Training Stupid Backoff (alpha=0.25) model... Done
Training Stupid Backoff (alpha=0.25) model... Done


# Results

After training our various models, we now need to determine the validity of our word predictor models and see how they shape up against sentences they have never seen. First we prepare our test data, which begins by tokenizing and normalizing in the exact same manner as the training data. We then mask one word token randomly in each sentence token, adding that now masked word to a list, and then check if the word generated by each model is an exact match to the masked word. We then report the exact accuracy of each model as well as the average number of keystrokes saved.

In [8]:
sentence_tokens = []
with open('../data/sample_test.csv') as csv_file:
    print('Tokenizing and normalizing testing data...', end=' ')
    sentence_tokens = tokenize_doc(csv_file)
    print('Done')

print('Masking word token in each sentence token...', end=' ')
masked_sentence_tokens, masked_words = mask_tokens(sentence_tokens.copy())
print('Done')

Tokenizing and normalizing testing data... Done
Masking word token in each sentence token... Done


In [9]:
import statistics

print('Total Testing Masked Sentence Tokens:', len(masked_sentence_tokens))
print('Average Number of Testing Word Tokens per Testing Masked Sentence Token:', statistics.mean(map(lambda e: len(e), masked_sentence_tokens)))
print('Example Testing Masked Sentence Token:', masked_sentence_tokens[0])

Total Testing Masked Sentence Tokens: 2366
Average Number of Testing Word Tokens per Testing Masked Sentence Token: 18.664412510566358
Example Testing Masked Sentence Token: ['ok', ',', 'i', 'read', 'that', 'the', '1990', 'version', 'had', 'nudity', ',', 'so', 'i', 'thought', 'since', 'this', 'said', 'it', 'was', 'in', '1970', 'something', 'it', 'was', 'safe', ',', 'and', 'than', 'the', 'rating', 'said', 'g', ',', 'i', 'had', 'to', 'fast', 'forward', 'at', 'least', '1/3', 'of', '<MASK>', 'movie', ',', 'and', 'finally', 'just', 'decided', 'it', 'was', "n't", 'even', 'worth', 'it', ',', 'and', 'just', 'cut', 'it', 'off', '!']


## N-Gram Models

### Maximum Likelihood Estimator (MLE)

In [10]:
print('Testing MLE model...', end=' ')
mle_exact_accuracy, mle_avg_keystrokes_saved = test_model(mle, masked_sentence_tokens, masked_words)
print('Done')

Testing MLE model... Done


In [11]:
print('MLE Exact Prediction Accuracy:', mle_exact_accuracy, '%')
print('MLE Average Keystrokes Saved:', mle_avg_keystrokes_saved)

MLE Exact Prediction Accuracy: 9.382924767540151 %
MLE Average Keystrokes Saved: 2.7387387387387387


The MLE model is able to push out decent performance compared to the other models. While the accuracy seems low, we must consider that we are looking for exact matches, which does not take into account misspelled masked words or the relevancy of a word predicted. Our model could predict a correctly spelled version of the misppelled masked word but it would be considered wrong. Predicting "book" instead of "novel" would also be considered wrong, but is the model necessarily wrong for predicting the word "book"? No, but we will address this in Discussiona & Future Work. From the numbers, we can also see that the model averages around 2 - 3 keystrokes saved whcih means it perform better on words like "the" or "and" which may be indicative of a lack of diverse words in our training data.

### Laplace

In [12]:
print('Testing Laplace model...', end=' ')
laplace_exact_accuracy, laplace_avg_keystrokes_saved = test_model(laplace, masked_sentence_tokens, masked_words)
print('Done')

Testing Laplace model... Done


In [13]:
print('Laplace Exact Prediction Accuracy:', laplace_exact_accuracy, '%')
print('Laplace Average Keystrokes Saved:', laplace_avg_keystrokes_saved)

Laplace Exact Prediction Accuracy: 7.776838546069316 %
Laplace Average Keystrokes Saved: 2.652173913043478


The Laplace model performs poorly compared to the other models which we found to be interesting considering it should have a more general word prediction accuracy. We see comparable average keystrokes saved to other models so while it is getting more predictions wrong, it is getting similar structured words.

### Lidstone

In [14]:
print('Testing Lidstone models...')

print('Testing Lidstone (gamma=0.25) model...', end=' ')
lidstone_25_exact_accuracy, lidstone_25_avg_keystrokes_saved = test_model(lidstone_25, masked_sentence_tokens, masked_words)
print('Done')

print('Testing Lidstone (gamma=0.50) model...', end=' ')
lidstone_50_exact_accuracy, lidstone_50_avg_keystrokes_saved = test_model(lidstone_50, masked_sentence_tokens, masked_words)
print('Done')

print('Testing Lidstone (gamma=0.75) model...', end=' ')
lidstone_75_exact_accuracy, lidstone_75_avg_keystrokes_saved = test_model(lidstone_75, masked_sentence_tokens, masked_words)
print('Done')

Testing Lidstone models...
Testing Lidstone (gamma=0.25) model... Done
Testing Lidstone (gamma=0.50) model... Done
Testing Lidstone (gamma=0.75) model... Done


In [19]:
print('Lidstone (gamma=0.25) Exact Prediction Accuracy:', lidstone_25_exact_accuracy, '%')
print('Lidstone (gamma=0.50) Exact Prediction Accuracy:', lidstone_50_exact_accuracy, '%')
print('Lidstone (gamma=0.75) Exact Prediction Accuracy:', lidstone_75_exact_accuracy, '%')
print('Lidstone (gamma=0.25) Average Keystrokes Saved:', lidstone_25_avg_keystrokes_saved)
print('Lidstone (gamma=0.50) Average Keystrokes Saved:', lidstone_50_avg_keystrokes_saved)
print('Lidstone (gamma=0.75) Average Keystrokes Saved:', lidstone_75_avg_keystrokes_saved)

Lidstone (gamma=0.25) Exact Prediction Accuracy: 8.664412510566356 %
Lidstone (gamma=0.50) Exact Prediction Accuracy: 9.08706677937447 %
Lidstone (gamma=0.75) Exact Prediction Accuracy: 8.030431107354184 %
Lidstone (gamma=0.25) Average Keystrokes Saved: 2.6682926829268294
Lidstone (gamma=0.50) Average Keystrokes Saved: 2.6232558139534885
Lidstone (gamma=0.75) Average Keystrokes Saved: 2.7157894736842105


The Lidstone models performed decently compared to the other models. The sweet spot seems to be a gamma of 0.5 with a smoothing level closer to 1 (Laplace model) showing decreased accuracy and closer to 0 (MLE model) also showing decreased performance but not as significant. Average keystrokes saved is steady for the models considered thus far.

### Stupid Backoff

In [18]:
print('Testing Stupid Backoff models...')

print('Testing Stupid Backoff (alpha=0.25) model...', end=' ')
stupid_backoff_25_exact_accuracy, stupid_backoff_25_avg_keystrokes_saved = test_model(stupid_backoff_25, masked_sentence_tokens, masked_words)
print('Done')

print('Testing Stupid Backoff (alpha=0.50) model...', end=' ')
stupid_backoff_50_exact_accuracy, stupid_backoff_50_avg_keystrokes_saved = test_model(stupid_backoff_50, masked_sentence_tokens, masked_words)
print('Done')

print('Testing Stupid Backoff (alpha=0.75) model...', end=' ')
stupid_backoff_75_exact_accuracy, stupid_backoff_75_avg_keystrokes_saved = test_model(stupid_backoff_75, masked_sentence_tokens, masked_words)
print('Done')

Testing Stupid Backoff models...
Testing Studid Backoff (alpha=0.25) model... Done
Testing Stupid Backoff (alpha=0.50) model... Done
Testing Stupid Backoff (alpha=0.75) model... Done


In [20]:
print('Stupid Backoff (alpha=0.25) Exact Prediction Accuracy:', stupid_backoff_25_exact_accuracy, '%')
print('Stupid Backoff (alpha=0.50) Exact Prediction Accuracy:', stupid_backoff_50_exact_accuracy, '%')
print('Stupid Backoff (alpha=0.75) Exact Prediction Accuracy:', stupid_backoff_75_exact_accuracy, '%')
print('Stupid Backoff (alpha=0.25) Average Keystrokes Saved:', stupid_backoff_25_avg_keystrokes_saved)
print('Stupid Backoff (alpha=0.50) Average Keystrokes Saved:', stupid_backoff_50_avg_keystrokes_saved)
print('Stupid Backoff (alpha=0.75) Average Keystrokes Saved:', stupid_backoff_75_avg_keystrokes_saved)

Stupid Backoff (alpha=0.25) Exact Prediction Accuracy: 8.495350803043111 %
Stupid Backoff (alpha=0.50) Exact Prediction Accuracy: 9.04480135249366 %
Stupid Backoff (alpha=0.75) Exact Prediction Accuracy: 9.721048182586644 %
Stupid Backoff (alpha=0.25) Average Keystrokes Saved: 2.6218905472636815
Stupid Backoff (alpha=0.50) Average Keystrokes Saved: 2.822429906542056
Stupid Backoff (alpha=0.75) Average Keystrokes Saved: 2.8043478260869565


The Stupid Backoff models seem to consistently perform the best out of our models for our problem. An alpha level of 0.75 seemed to show the best accuracy which means that the more lower probability words affect the predictions, the better the accuracy for exact predictions. This is also indicative that we would need to train on more sentences to possibly get better representative probabilities for our word predictions. This also coincides with the fact that each of our models have relatively the same average keystrokes saved.

# Discussion & Future Work